In [8]:
from collections import deque

import numpy as np

import torch

import random

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

#import lightning as L
from pytorch_lightning import Trainer

#import qnet

In [1]:
!pip install mido

!pip install pytorch_lightning

# !pip install qnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
test_midis = ['monteverdi_libri_dei_madrigali_1_5_(c)icking-archive.mid',
              'monteverdi_libri_dei_madrigali_1_10_(c)icking-archive.mid',
              'monteverdi_libri_dei_madrigali_4_12_(c)icking-archive.mid',
              'monteverdi_libri_dei_madrigali_4_13_(c)icking-archive.mid']

In [5]:
import os
from mido import MidiFile

input_folder = 'path/to/your/midi/folder'
output_folder = 'path/to/your/output/folder'

# Make sure the output folder exists
os.makedirs(output_folder, exist_ok=True)

def Midi_File_Input(mids):
    token = []
    for i, track in enumerate(mids.tracks):
        for msg in track:
            if msg.type == 'note_on':
                token.append(f"<{msg.type}_{msg.channel}_{msg.note}_{msg.velocity}>")
            elif msg.type == 'note_off':
                token.append(f"<{msg.type}_{msg.channel}_{msg.note}_{msg.velocity}>")
            # elif msg.type == 'track_name':
            #     token.append(f"<{msg.type}_{msg.name}>")
            elif msg.type == 'control_change':
                token.append(f"<{msg.type}_{msg.channel}_{msg.control}_{msg.value}>")
            elif msg.type == 'program_change':
                token.append(f"<{msg.type}_{msg.program}_>")
            elif msg.type == 'key_signature':
                token.append(f"<{msg.type}_{msg.key}>")
    return token

token_data = {}
all_tokens = []  # Flat list for training


# Process each MIDI file
for filename in test_midis:
    midi_path = '/content/drive/MyDrive/Final Project Folder/Midi Files/' + filename

    midi = MidiFile(midi_path)

    tokens = Midi_File_Input(midi)

    # Add song_start and song_end markers
    song_tokens = ['<song_start>'] + tokens + ['<song_end>']

    # Store in dictionary (per-song)
    token_data[filename] = song_tokens

    # Append to flat list (for generative model training)
    all_tokens.extend(song_tokens)


In [6]:
from collections import Counter

# Flatten all tokens across all songs
all_tokens_flat = [token for tokens in token_data.values() for token in tokens]

# Count and sort tokens (optional for ordering)
token_freq = Counter(all_tokens_flat)

# Assign token ID
vocab = {token: idx for idx, token in enumerate(sorted(token_freq))}

# Optionally store reverse map too:
inv_vocab = {idx: token for token, idx in vocab.items()}

token_ids_data = {}

for filename, tokens in token_data.items():
    token_ids = [vocab[token] for token in tokens]
    token_ids_data[filename] = token_ids

vocab["<pad>"] = 279

{0: '<control_change_0_100_0>',
 1: '<control_change_0_101_0>',
 2: '<control_change_0_10_104>',
 3: '<control_change_0_10_24>',
 4: '<control_change_0_10_44>',
 5: '<control_change_0_10_64>',
 6: '<control_change_0_10_84>',
 7: '<control_change_0_12_3>',
 8: '<control_change_0_38_0>',
 9: '<control_change_0_6_12>',
 10: '<control_change_0_7_100>',
 11: '<control_change_0_7_101>',
 12: '<control_change_0_7_102>',
 13: '<control_change_0_7_103>',
 14: '<control_change_0_7_104>',
 15: '<control_change_0_7_105>',
 16: '<control_change_0_7_106>',
 17: '<control_change_0_7_107>',
 18: '<control_change_0_7_108>',
 19: '<control_change_0_7_109>',
 20: '<control_change_0_7_110>',
 21: '<control_change_0_7_111>',
 22: '<control_change_0_7_112>',
 23: '<control_change_0_7_113>',
 24: '<control_change_0_7_114>',
 25: '<control_change_0_7_115>',
 26: '<control_change_0_7_116>',
 27: '<control_change_0_7_117>',
 28: '<control_change_0_7_118>',
 29: '<control_change_0_7_83>',
 30: '<control_change_0

Positional Encoding

In [9]:
class PositionalEncoding(nn.Module):
  def __init__(self,d_model, max_length):

    super().__init__()

    pe = torch.zeros(max_length, d_model) #Here we start by creating an empty matrix which will get updated.

  #d_model is the number of word embedding dimensions the larger this is the richer our description of a word
  #The actual word embedding values are updated via back propagation

    position = torch.arange(start = 0, end = max_length, step = 1).float().unsqueeze(1)

    embedding_index = torch.arange(start=0, end=d_model,step=2).float()

    div_term = 1/torch.tensor(10000.0)**(embedding_index/ d_model)


    #Our Positional encoding starts with a sin and cos equation.
  #Have a look at the literture to check why we do this but it's not too difficult to understand

    pe[:, 0::2] = torch.sin(position * div_term) #This updates the first column
    pe[:, 1::2] = torch.cos(position * div_term) # This updates the second column

  #This happens in an alternating method where the first column and the secondary column gets updated

    self.register_buffer('pe', pe.unsqueeze(0))

  def forward (self, word_embeddings):

    seq_len = word_embeddings.size(1)
    return word_embeddings + self.pe[:, :seq_len, :]

    #return word_embeddings + self.pe[:word_embeddings.size(0), :] #Finally we add the positonal encoding values to the word embedding values

Attention Coding

In [10]:
class Attention(nn.Module):
  def __init__(self, d_model):

#Here we pass in the number of word embedding values per token This is for the sake of matrix nultiplication
#When we do our matrix multiplication to create the queries, keys and values.

    super().__init__()

    self.W_q = nn.Linear(in_features = d_model, out_features = d_model, bias = False)
    self.W_k = nn.Linear(in_features = d_model, out_features = d_model, bias = False)
    self.W_v = nn.Linear(in_features = d_model, out_features = d_model, bias = False)

    #Above we have our weight matrix which are used to calculate our Query, Keys and Values
    #Finally, we don't include a bias term here when calculating attention which is why we set the bias to False

    self.row_dim = 0
    self.col_dim = 1 #To give us flexibility to input data sequentiallly into batches we create variables to do so

  def forward(self, encodings_for_q, encodings_for_k, encodings_for_v, mask = None):

      #The forward method is where we use the masked self attention values for each token to be calculated
      #Next we give this forward method the flexbility
      #Finally since we want to do masked self attention we can pass in a mask too.

    q = self.W_q(encodings_for_q)
    k = self.W_k(encodings_for_k)
    v = self.W_v(encodings_for_v)

    #sims = torch.matmul(q, k.transpose(dim0 = self.row_dim, dim1 = self.col_dim))

    #scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)

    sims = torch.matmul(q, k.transpose(-2, -1))  # [batch_size, seq_len, seq_len]

    scaled_sims = sims / (k.size(-1) ** 0.5)

    if mask is not None:
      scaled_sims = scaled_sims.masked_fill(mask = mask, value=-1e9) #Here we include our mask values which stops the attention from looking ahead to caluclate weights.

    attention_percents = F.softmax(scaled_sims, dim = self.col_dim)

    attention_scores = torch.matmul(attention_percents, v)

    return attention_scores

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

#from pytorch_lightning import
from pytorch_lightning import Trainer, LightningModule


class DecodeOTrans(LightningModule):
  def __init__(self, d_model, max_length, num_tokens = 100):

    #Here we specify the number of tokens available in the vocabulary,
    #Number of word embeddings per token
    #Max token length
    super().__init__()

    self.we = nn.Embedding(num_embeddings = num_tokens, embedding_dim = d_model) #We creating a word embedding value
    #Embeddings needs to know how many tokens are in the vocabulary and understand the dimension size to represent the embedding

    self.pe = PositionalEncoding(d_model = d_model, max_length = max_length)

    #Then we create a positional encoding object using the class we created earlier

    self.self_attention = Attention(d_model = d_model)

    self.fc_layer = nn.Linear(in_features = d_model, out_features = num_tokens) #This is our fully connected layer also know as our Dense Layer of neurons (RNN neural networks)

    self.loss = nn.CrossEntropyLoss()

  def encode_state(self, token_ids):

    word_embeddings = self.we(token_ids)  # [batch_size, seq_len, d_model]
    positional_encoding = self.pe(word_embeddings)  # add positional encoding

    seq_len = token_ids.size(1)
    #mask = torch.tril(torch.ones((seq_len, seq_len), device=token_ids.device)) == 0

    mask = torch.tril(torch.ones((seq_len, seq_len), device=token_ids.device)).unsqueeze(0) == 0

    self_attention_values = self.self_attention(
        positional_encoding,
        positional_encoding,
        positional_encoding,
        mask=mask
    )

    contextual_embeddings = positional_encoding + self_attention_values

    # Option A: use last token's hidden state
    state_vector = contextual_embeddings[:, -1, :]

    # OR Option B: use mean pooling
    # state_vector = contextual_embeddings.mean(dim=1)

    return state_vector  # [batch_size, d_model]

  def forward(self, token_ids):
    return self.encode_state(token_ids)

#Difference between state vectors and probability logits in a sentence
# The state vector is a rich, high-dimensional summary of the input sequence
#  (via embeddings + position + attention),
#  and the logits are what you get after mapping that state vector
#  through a final linear layer into output space
#   (like actions or vocabulary).

This is my Practice exploratory code

In [13]:
x = DecodeOTrans(d_model=128, max_length=20, num_tokens=len(vocab))


In [19]:
sample_input = torch.randint(0, len(vocab), (1, 10))

#The torch.randit creates an empty matrix to place my state vector values into

state_vector = x(sample_input)





NameError: name 'state_vecotr' is not defined

In [23]:
sample_input

tensor([[ 77, 133, 106,  35,  79,  28,  42, 139, 259,  97]])

Above is my state vector  we get the (batch_size, d_model) where d_model is the size of the embeddings so each token gets the same number of numebrs which represent the token.

In [26]:
class QNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_actions):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, num_actions)

    def forward(self, state_vector):
        x = F.relu(self.linear1(state_vector))
        return self.linear2(x)  # Q-values for each action


Agent RL + Trans (For Music GPT Adapted)

In [27]:
class MusicDQNAgent:
    def __init__(self, d_model, max_length, num_tokens, num_actions):

      self.encoder = DecodeOTrans(d_model, max_length, num_tokens)
      #This is our transformer based encoder we created which takes the tokens and outputs it as dense vectors with positional + attentional mechanism converted into a simple numerical format
      self.q_net = QNet(d_model, 128, num_actions)
      # This is the Q neural network this predicts possible actions from the inputted state.

      self.gamma = 0.99

      #Hyperparameter tells the model to reward future or immediate rewards.
      self.epsilon = 0.1

      #This is the exploration rate and tells the model to explore so choose randomly to further explore the entire environment and not get stuck at local minima.
      #self.memory = deque(maxlen=10000)

      #This stores tuples of (state, action, reward, next_state, done).
      self.optimizer = torch.optim.Adam(list(self.encoder.parameters()) + list(self.q_net.parameters()), lr=1e-4)

      #This is our optimiser Adam here we're balancing the predicted and Q values.

      #Parameters

        #d_model: The dimensionality of each token vector the larger the dimensionality the more detail which is included in the tokens descriptions
        #max_length: maximum sequence length expected by the transformer
        #num_tokens: Total number of unique tokens in my vocabulary
        #num_actions: The number of possible actions the agent can choose from
    def get_state(self, token_sequence, no_grad=True):

        # token_sequence: [1, seq_len]
      return self.encoder(token_sequence) # [1, d_model]


    #Here we are defining how we are going to grab the state we are in

    def get_action(self, state_vector):
      if random.random() < self.epsilon:
        #This generates a float between 0 and 1 if the epison is larger then it will got with epsilon
        #Therefore the larger the episoln the less exploration there is
          return random.randint(0, self.q_net.linear2.out_features - 1)
            #If we are exploring then return a random token where num_tokens -1 would be the next one.
      q_values = self.q_net(state_vector)
        #If we are not exploring then use the current state to predict the next state using the Q Net.
      return torch.argmax(q_values).item()

    #     #Here we choose the token with the highest indexed value
    # def remember(self, state, action, reward, next_state, done):

    #   self.memory.append((state, action, reward, next_state, done))

    # def train_short_memory(self, state, action, reward, next_state, done):
    #     self.train_step(state, action, reward, next_state, done)

    # def train_long_memory(self, batch_size=64):
    #     if len(self.memory) < batch_size:
    #         return

    #     mini_batch = random.sample(self.memory, batch_size)

    #     for state, action, reward, next_state, done in mini_batch:
    #         self.train_step(state, action, reward, next_state, done)

    def compute_reward(predicted_token, target_token):

      return 1.0 if predicted_token == target_token else -0.1

    def train_step(self, state, action, reward, next_state, done):

      self.q_net.train()

        #This puts the Q neural network into training mode

      q_values = self.q_net(state)

        #So forward passes through the Q network
      target = q_values.clone().detach()

        #Create a copy of the Q-values tensor.

        #Detach() removes it from the computation graph — this prevents gradients from flowing through the target.

      next_q = self.q_net(next_state)
      q_target = reward + self.gamma * torch.max(next_q) * (1 - int(done))

        #We're computing the target Q-value using the Bellman equation:

      target[0, action] = q_target

        #We now replace only the Q-value for the action we actually took with the target value.

        #This tells the network:

        #"You predicted X for action A, but the real value should have been Y — update your weights accordingly."

      loss = F.mse_loss(q_values, target)
      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()

        #Mean Squared Error between:

        #q_values: what the network predicted

        #target: what it should have predicted (Bellman update)

        #Then we run standard backprop and gradient descent.

In [37]:
for index in test_midis:
  print(index)

monteverdi_libri_dei_madrigali_1_5_(c)icking-archive.mid
monteverdi_libri_dei_madrigali_1_10_(c)icking-archive.mid
monteverdi_libri_dei_madrigali_4_12_(c)icking-archive.mid
monteverdi_libri_dei_madrigali_4_13_(c)icking-archive.mid


In [45]:
input_sequences = []


for i in test_midis:
  print(token_ids_data[i])

[277, 48, 275, 28, 28, 28, 28, 28, 2, 6, 5, 4, 3, 46, 7, 46, 7, 46, 7, 46, 7, 46, 7, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 13, 13, 13, 13, 248, 221, 203, 126, 14, 14, 14, 14, 16, 16, 16, 16, 18, 18, 18, 18, 20, 20, 20, 20, 22, 22, 22, 22, 24, 24, 24, 24, 26, 26, 26, 26, 27, 27, 27, 27, 28, 28, 28, 28, 27, 27, 27, 27, 26, 26, 26, 26, 79, 75, 72, 60, 26, 26, 26, 26, 248, 216, 194, 167, 27, 27, 27, 27, 28, 28, 28, 28, 27, 27, 27, 27, 26, 26, 26, 26, 79, 74, 71, 67, 272, 220, 202, 170, 84, 75, 72, 68, 253, 236, 203, 156, 80, 77, 72, 65, 247, 247, 202, 146, 79, 79, 72, 64, 26, 26, 26, 26, 253, 236, 203, 156, 27, 27, 27, 27, 28, 28, 28, 28, 27, 27, 27, 27, 26, 26, 26, 26, 80, 77, 72, 65, 26, 26, 26, 26, 248, 216, 194, 167, 27, 27, 27, 27, 28, 28, 28, 28, 27, 27, 27, 27, 26, 26, 26, 26, 79, 74, 71, 67, 266, 248, 221, 140, 82, 79, 75, 63, 263, 241, 218, 137, 82, 79, 75, 63, 262, 240, 217, 136, 82, 79, 75, 63, 266, 248, 221, 140, 82, 79, 75, 63, 252, 235, 202, 155, 80, 77,

In [31]:
agent = MusicDQNAgent(d_model=128, max_length=20, num_tokens=len(vocab), num_actions=len(vocab))
num_episodes = 500

for index in test_midis:
  target_sequence_list = token_ids_data[index]
  for episode in range(num_episodes):
    current_seq = [vocab["<song_start>"]]
    done = False

    while not done and len(current_seq) < 20:
        state_tensor = torch.tensor([current_seq], dtype=torch.long)

        # Ensure state_tensor sequence length does not exceed max_length
        if state_tensor.shape[1] > agent.encoder.pe.pe.shape[1]:
            break # Stop if the sequence gets too long


        state_vector = agent.get_state(state_tensor, no_grad=False)

        # Detach the state vector for action selection as we don't need gradients here
        action = agent.get_action(state_vector.detach())


        next_seq = current_seq + [action]
        next_state_tensor = torch.tensor([next_seq], dtype=torch.long)

        # Determine if the episode is done and calculate reward
        done = action == vocab["<song_end>"] or len(next_seq) >= 20

        # Calculate reward, handling the case where we go beyond the target sequence length
        if len(current_seq) < len(target_sequence_list):
            reward = 1.0 if action == target_sequence_list[len(current_seq)] else -0.1
        else:
            # Penalize if we exceed the target length without generating the end token
            reward = -0.5 if action != vocab["<song_end>"] else 1.0


        # Get next_state_vector only if not done to avoid potential issues with sequence length if done
        if not done and next_state_tensor.shape[1] <= agent.encoder.pe.pe.shape[1]:
             next_state_vector = agent.get_state(next_state_tensor, no_grad=False)
        else:
             next_state_vector = None # No next state to consider if done or exceeding max length


        # Train the agent
        # Pass next_state_vector to train_step even if it's None, the train_step will handle it
        agent.train_step(state_vector, action, reward, next_state_vector, done)


        current_seq = next_seq

TypeError: linear(): argument 'input' (position 1) must be Tensor, not NoneType

Deepseek stripped back version

In [28]:
test_midi = ['monteverdi_libri_dei_madrigali_1_5_(c)icking-archive.mid']

In [42]:
agent = MusicDQNAgent(d_model=128, max_length=20, num_tokens=len(vocab), num_actions=len(vocab))
num_episodes = 500

for index in test_midi:
    target_sequence = token_ids_data[index]  # Target sequence for this episode

    #target_sequence = torch.tensor(token_ids_data)
    for episode in range(num_episodes):
        current_seq = [vocab["<song_start>"]]  # Initialize with start token
        done = False

        while not done and len(current_seq) < 20:
            # Convert current sequence to tensor (shape: [1, seq_len])
            state_tensor = torch.tensor([current_seq], dtype=torch.long)

            # Get state vector (with gradients for training)
            state_vector = agent.get_state(state_tensor, no_grad=False)

            # Select action (no gradients needed here)
            action = agent.get_action(state_vector.detach())

            # Update sequence
            next_seq = current_seq + [action]
            done = (action == vocab["<song_end>"]) or (len(next_seq) >= 20)

            # Calculate reward
            if len(current_seq) < len(target_sequence):
                reward = 1.0 if (action == target_sequence[len(current_seq)]) else -0.1
            else:
                reward = -0.5 if (action != vocab["<song_end>"]) else 1.0

            # Get next state vector if needed
            if not done:
                next_state_tensor = torch.tensor([next_seq], dtype=torch.long)
                next_state_vector = agent.get_state(next_state_tensor, no_grad=False)
            else:
                next_state_vector = torch.zeros_like(state_vector)

            # Train the agent
            agent.train_step(state_vector, action, reward, next_state_vector, done)

            current_seq = next_seq

In [45]:
current_seq

[277,
 211,
 275,
 173,
 173,
 252,
 173,
 206,
 173,
 173,
 202,
 173,
 193,
 173,
 131,
 173,
 13,
 265,
 173,
 173]

In [41]:
type(next_state_vector)

NoneType

In [40]:
next_seq

[277,
 243,
 263,
 132,
 62,
 164,
 243,
 166,
 166,
 166,
 166,
 166,
 155,
 123,
 19,
 135,
 166,
 166,
 166,
 153]

In [43]:
import torch

# Assume 0 corresponds to <start> in your vocabulary
start_token_id = 277

# Begin with just the start token
generated_sequence = [start_token_id]

max_length = 20  # Set how long you want the output to be

for _ in range(max_length):
    # Convert to tensor: [1, seq_len]
    token_tensor = torch.tensor([generated_sequence], dtype=torch.long)

    # Get encoded state from transformer encoder
    state_vector = agent.get_state(token_tensor)

    # Choose next token (action)
    next_token = agent.get_action(state_vector)

    # Optionally stop if end token is produced (e.g., <end> = 1)
    if next_token == 1:  # Assuming 1 is <end>
        break

    # Add predicted token to sequence
    generated_sequence.append(next_token)

# Decode tokens back to symbols if needed
print("Generated token sequence:", generated_sequence)


Generated token sequence: [277, 211, 275, 173, 233, 173, 173, 6, 173, 173, 49, 273, 173, 72, 173, 173, 68, 173, 173, 92, 173]


Having a look at my generated sequence

In [50]:
named_sequence = [inv_vocab[token_id] for token_id in generated_sequence]

In [51]:
named_sequence

['<song_start>',
 '<note_on_0_69_119>',
 '<program_change_52_>',
 '<note_on_0_64_116>',
 '<note_on_0_72_124>',
 '<note_on_0_64_116>',
 '<note_on_0_64_116>',
 '<control_change_0_10_84>',
 '<note_on_0_64_116>',
 '<note_on_0_64_116>',
 '<note_off_0_43_0>',
 '<note_on_0_79_127>',
 '<note_on_0_64_116>',
 '<note_off_0_67_0>',
 '<note_on_0_64_116>',
 '<note_on_0_64_116>',
 '<note_off_0_63_0>',
 '<note_on_0_64_116>',
 '<note_on_0_64_116>',
 '<note_on_0_47_127>',
 '<note_on_0_64_116>']

Agent (Snakes Game)

In [56]:
import mido
from mido import Message, MidiFile, MidiTrack
import re
from google.colab import files

def tokens_to_midi(token_sequence, output_path='output.mid'):
    # Create a new MIDI file with one track
    midi = MidiFile()
    track = MidiTrack()
    midi.tracks.append(track)

    # Process each token in sequence
    for token in token_sequence:
        # Skip song start token
        if token == '<song_start>':
            continue

        # Extract event details using regex
        match = re.match(r'<(.*?)_(.*?)>', token)
        if not match:
            continue

        event_type = match.group(1)
        params = match.group(2).split('_')

        # Handle different event types
        if event_type == 'note_on':
            channel = int(params[0])
            note = int(params[1])
            velocity = int(params[2])
            track.append(Message('note_on', channel=channel, note=note, velocity=velocity))

        elif event_type == 'note_off':
            channel = int(params[0])
            note = int(params[1])
            velocity = int(params[2])
            track.append(Message('note_off', channel=channel, note=note, velocity=velocity))

        elif event_type == 'control_change':
            channel = int(params[0])
            control = int(params[1])
            value = int(params[2])
            track.append(Message('control_change', channel=channel, control=control, value=value))

        elif event_type == 'program_change':
            program = int(params[0])
            track.append(Message('program_change', program=program))

        elif event_type == 'key_signature':
            key = params[0]
            # Key signature meta message
            track.append(mido.MetaMessage('key_signature', key=key))

    # Save the MIDI file
    midi.save(output_path)
    return midi

# Example usage:

midi = tokens_to_midi(named_sequence, 'output.mid')
files.download('output.mid')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>